In [1]:
import os

os.chdir("/Users/abhisheksrivastava/Documents/LLM-end-to-end-exploration/my_multi_agent_project")
print("CWD:", os.getcwd())

CWD: /Users/abhisheksrivastava/Documents/LLM-end-to-end-exploration/my_multi_agent_project


In [2]:
### Run with MLflow tracking

In [ ]:
from langgraph_rag.ingest_runner import run_ingestion_with_mlflow
from langgraph_rag.config import IngestionConfig, TableConfig

config = IngestionConfig(
    tables=[
        TableConfig(
            name="patients",
            path="data/patients.txt",
            id_column="PATIENT_ID",
            text_columns=None,
        ),
    ],
    metadata_text_path="data/clinical_patient_meta.txt",
    persist_dir="chroma_db",
    collection_name="patient_docs",
    chunk_token_size=800,
    chunk_overlap=400,
)

collection, run_id = run_ingestion_with_mlflow(config)
collection.count(), run_id


/Users/abhisheksrivastava/Documents/LLM-end-to-end-exploration/my_multi_agent_project/.venv/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)


In [ ]:
### Manual Run

In [5]:
from langgraph_rag.config import IngestionConfig, TableConfig
from langgraph_rag.chunking import build_chunked_documents
from langgraph_rag.vectorstore import build_persistent_vector_store

config = IngestionConfig(
    tables=[
        TableConfig(
            name="patients",
            path="data/patients.txt",
            id_column="PATIENT_ID",
            text_columns=None,
        ),
    ],
    metadata_text_path="data/clinical_patient_meta.txt",
    persist_dir="chroma_db",
    collection_name="patient_docs",
    embedding_model="text-embedding-3-large",
    chunk_token_size=800,
    chunk_overlap=400,
)


In [6]:
chunks = build_chunked_documents(config)
len(chunks), chunks[0]


(24957,
 DocChunk(id='patients:P-0000012:0:chunk_0', text='GENDER: Female\nRACE: White\nETHNICITY: Non-Spanish; Non-Hispanic\nCURRENT_AGE_DEID: 68.0\nSTAGE_HIGHEST_RECORDED: Stage 1-3\nNUM_ICDO_DX: 2\nADRENAL_GLANDS: No\nBONE: No\nCNS_BRAIN: No\nINTRA_ABDOMINAL: Yes\nLIVER: No\nLUNG: Yes\nLYMPH_NODES: Yes\nOTHER: Yes\nPLEURA: No\nREPRODUCTIVE_ORGANS: No\nSMOKING_PREDICTIONS_3_CLASSES: Former/Current Smoker\nHISTORY_OF_PDL1: No\nPRIOR_MED_TO_MSK: Unknown\nOS_MONTHS: 118.4546647\nOS_STATUS: 0:LIVING\nHR: No\nHER2: No', metadata={'source': 'patient_table', 'table_name': 'patients', 'id_column': 'PATIENT_ID', 'row_id': 'P-0000012', 'row_index': 0, 'chunk_index': 0}))

In [4]:
collection = build_persistent_vector_store(config)
collection.count()


25085

In [7]:
collection.query(
    query_texts=["Display all the information"],
    n_results=10,
    where={"table_name": "patients"}
)


{'ids': [['patients:P-0021230:7772:chunk_0',
   'patients:P-0019084:6983:chunk_0',
   'patients:P-0006074:2106:chunk_0',
   'patients:P-0014928:5380:chunk_0',
   'patients:P-0014488:5179:chunk_0',
   'patients:P-0017824:6529:chunk_0',
   'patients:P-0014781:5308:chunk_0',
   'patients:P-0011250:4101:chunk_0',
   'patients:P-0007888:2840:chunk_0',
   'patients:P-0019784:7249:chunk_0']],
 'embeddings': None,
 'documents': [['Study ID: msk_chord_2024\nSample ID: P-0021230-T01-IM6\nTumor Site: Adrenal Glands (NLP): Unknown\nTumor Site: Bone (NLP): Unknown\nCancer Type: Breast Cancer\nCancer Type Detailed: Breast Invasive Cancer, NOS\nClinical Group: 1A\nClinical Summary: Localized\nTumor Site: CNS/Brain (NLP): Unknown\nCurrent Age: 63.0\nDiagnosis Description: Breast\nEthnicity: Non-Spanish; Non-Hispanic\nFraction Genome Altered: 0.1313\nSex: Female\nGene Panel: IMPACT468\nHER2: Yes\nHR: No\nICD-O Histology Description: Infiltrating Duct Carcinoma\nTumor Site: Intra Abdominal: Unknown\nTum

In [8]:
res = collection.get(
    ids=None,
    limit=5,
    include=["documents", "metadatas"],
)

len(res["ids"]), res["ids"][:3]


(5,
 ['patients:P-0000012:0:chunk_0',
  'patients:P-0000012:1:chunk_0',
  'patients:P-0000015:2:chunk_0'])

In [13]:
query = "stage IV lung cancer"
res = collection.query(
    query_texts=[query],
    n_results=3,
    include=["documents", "metadatas", "distances", "embeddings"],
)

res["metadatas"][0], res["documents"][0][0][:400]


([{'chunk_index': 0,
   'source': 'patient_table',
   'table_name': 'patients',
   'row_id': 'P-0047124',
   'row_index': 15325,
   'id_column': 'Patient ID'},
  {'chunk_index': 0,
   'source': 'patient_table',
   'table_name': 'patients',
   'row_id': 'P-0037789',
   'row_index': 12784,
   'id_column': 'Patient ID'},
  {'chunk_index': 0,
   'id_column': 'Patient ID',
   'row_id': 'P-0018833',
   'source': 'patient_table',
   'table_name': 'patients',
   'row_index': 6895}],
 'Study ID: msk_chord_2024\nSample ID: P-0047124-T01-IM6\nTumor Site: Adrenal Glands (NLP): No\nTumor Site: Bone (NLP): No\nCancer Type: Non-Small Cell Lung Cancer\nCancer Type Detailed: Lung Squamous Cell Carcinoma\nClinical Group: 4A\nClinical Summary: Distant Metastases/Systemic Disease\nTumor Site: CNS/Brain (NLP): No\nCurrent Age: 70.0\nDiagnosis Description: Lung and Bronchus\nEthnicity: Non-Spanish; Non')

In [14]:
res

{'ids': [['patients:P-0047124:15325:chunk_0',
   'patients:P-0037789:12784:chunk_0',
   'patients:P-0018833:6895:chunk_0']],
 'embeddings': [array([[ 0.01307581,  0.02582896, -0.00811894, ...,  0.00350893,
          -0.01082525,  0.00030804],
         [ 0.01599848,  0.03193267, -0.00612297, ...,  0.00389516,
          -0.01127366,  0.0012475 ],
         [ 0.00821325,  0.02556654, -0.00503741, ...,  0.00691096,
          -0.01115341, -0.00482569]], shape=(3, 3072))],
 'documents': [['Study ID: msk_chord_2024\nSample ID: P-0047124-T01-IM6\nTumor Site: Adrenal Glands (NLP): No\nTumor Site: Bone (NLP): No\nCancer Type: Non-Small Cell Lung Cancer\nCancer Type Detailed: Lung Squamous Cell Carcinoma\nClinical Group: 4A\nClinical Summary: Distant Metastases/Systemic Disease\nTumor Site: CNS/Brain (NLP): No\nCurrent Age: 70.0\nDiagnosis Description: Lung and Bronchus\nEthnicity: Non-Spanish; Non-Hispanic\nFraction Genome Altered: 0.0144\nSex: Male\nGene Panel: IMPACT468\nICD-O Histology Descrip